<a href="https://colab.research.google.com/github/pankajr141/experiments/blob/master/Reasoning/Reinforcement/Reasoning%20%7BRL%7D%20-%205%3A%20Partially%20Observable%20MDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## References 

https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-6-partial-observability-and-deep-recurrent-q-68463e9aeefc

https://web.stanford.edu/class/aa228/reports/2018/final150.pdf


**How to solve video games ?** we certainly cant use the same alogright used previously as the previous algo require entire world knowledge to be loaded in order to work. This family is knows as MDP(Markov-Decision-Process) where current state is sufficient for making furture decision (eg. Grid ENV all the states and their Q values are known)


The env in which a single frame has only partial information is called as Partially Observable MDP. 

**How to solve these ?** 



*   <font color='red'>We can stack together some frames and pass to our n/w to learn - This is memory consumable and inefficient as their is a limit to the number of frames we can keep and also the experiance relay buffer will be huge. </font>
*   <font color='green'>We can use **RNN/LSTM/GRU** </font>. These network by default keep track of past observation through hidden states, so instead of stacking few frames, we can just pass 1 frame at a time.



### Setup

In [0]:
import multiprocessing
multiprocessing.cpu_count()

2

In [0]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install pyvirtualdisplay ffmpeg > /dev/null 2>&1
!apt-get install -y x11-utils > /dev/null 2>&1
!pip install piglet pyglet > /dev/null 2>&1
!pip install gym==0.14.0 > /dev/null 2>&1

In [4]:
!git clone https://github.com/openai/mujoco-py.git
!cd mujoco-py;pip3 install -r requirements.txt;python3 setup.py install

!apt-get install swig
!easy_install box2d
# !easy_install mujoco-py
# !easy_install gym[all]==0.14.0
!pip3 install gym[all]==0.14.0

Cloning into 'mujoco-py'...
remote: Enumerating objects: 1942, done.
remote: Total 1942 (delta 0), reused 0 (delta 0), pack-reused 1942
Receiving objects: 100% (1942/1942), 5.56 MiB | 21.41 MiB/s, done.
Resolving deltas: 100% (1192/1192), done.
  Created wheel for glfw: filename=glfw-1.8.3-cp36-none-any.whl size=23892 sha256=05e15b15329a4cef4b985cac7a29f3b50294f155526dca77d9912e3ba6828604
  Stored in directory: /root/.cache/pip/wheels/3a/d5/0e/2e3f5043f13868c6e3f857c7af4fa89d3bc331300ac0b5511b
Successfully built glfw
running install
running bdist_egg
running egg_info
creating mujoco_py.egg-info
writing mujoco_py.egg-info/PKG-INFO
writing dependency_links to mujoco_py.egg-info/dependency_links.txt
writing requirements to mujoco_py.egg-info/requires.txt
writing top-level names to mujoco_py.egg-info/top_level.txt
writing manifest file 'mujoco_py.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mujoco_py.egg-info/SOURCES.txt'
installing library code to b

In [1]:
''' Some utility functions '''
import glob
import io
import base64
from gym.wrappers import Monitor
from IPython.display import HTML
from IPython import display as ipythondisplay


def show_video():  
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1007'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1007'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

### Environment - CarRacing-v0

CarRacing env is a perfect example of POMDP, here we are trying to run a car on a randomly generated racetrack and we can see only a small portion of racetrack at a time. 


According to http://gym.openai.com/envs/CarRacing-v0/  -- official documentation

<pre>
Easiest continuous control task to learn from pixels, a top-down racing environment. Discreet control is reasonable in this environment as well, on/off discretisation is fine. State consists of 96x96 pixels. Reward is -0.1 every frame and +1000/N for every track tile visited, where N is the total number of tiles in track. For example, if you have finished in 732 frames, your reward is 1000 - 0.1*732 = 926.8 points. Episode finishes when all tiles are visited. Some indicators shown at the bottom of the window and the state RGB buffer. From left to right: true speed, four ABS sensors, steering wheel position, gyroscope.
</pre>

In [3]:
from IPython.display import HTML

HTML("""
    <video alt="test" controls>
        <source src="http://gym.openai.com/videos/2019-10-08--6QXvzzSWoV/CarRacing-v0/original.mp4" type="video/mp4">
    </video>
""")

`Action space is the set of triples (s, a, d) ∈ [−1, 1] × [0, 1] × [0, 1]`

<pre>
s -> steering coefficient ranges from hard left to hard right
a -> acceleration a ranges from none to full steam ahead
d -> deceleration d ranges from none to slamming the brakes.
</pre>

Each variable can take continous value however we can change our solution to include only discrete values which in turn will make the solution into a classification problem rather then regression.

<pre> A = {left, right, accelerate, decelerate, nothing} </pre> 

The above will accept 5 values in form of `[0, 1]` such that on or off. 




In [0]:
import gym
print("Action Space:", gym.make("CarRacing-v0").action_space.sample())

### Random Agent

Below is a random agent with continous  action space

In [6]:
!rm -rf video

import gym

env = wrap_env(gym.make("CarRacing-v0"))
env.reset()

steps = 1000
reward_sum = 0

for step in range(steps):
    env.render()
    observation, reward, done, _ = env.step(env.action_space.sample())
    reward_sum += reward
  
    if done:
        print("Reward for this episode was: {}, Dead at: {}".format(reward_sum, step))
        reward_sum = 0
        env.reset()
        break

env.close()
print("Reward for this episode was: {}, Dead at: {}".format(reward_sum, steps))

show_video()

Track generation: 1195..1498 -> 303-tiles track
Reward for this episode was: -33.774834437086646, Dead at: 999
Track generation: 1029..1296 -> 267-tiles track
Reward for this episode was: 0, Dead at: 1000


Below is a random agent with discrete action space

In [0]:
def convert_discreate_action_to_continous(left, right, accelerate, decelerate, nothing):
  if nothing == 1:
    return [0, 0, 0]
  elif left == 1:
    return [-1, 0, 0]
  elif right == 1:
    return [1, 0, 0]
  elif accelerate == 1:
    return [0, 1, 0]
  elif decelerate == 1:
    return [0, 0, 1]
  
def get_random_action():
  return convert_discreate_action_to_continous(*np.eye(5)[np.random.choice(5, 1)].tolist()[0])

In [13]:
!rm -rf video

import gym
import numpy as np

env = wrap_env(gym.make("CarRacing-v0"))
env.reset()

steps = 1000
reward_sum = 0

for step in range(steps):
    env.render()
    action = get_random_action()
    observation, reward, done, _ = env.step(action)
    reward_sum += reward
  
    if done:
        print("Reward for this episode was: {}, Dead at: {}".format(reward_sum, step))
        reward_sum = 0
        env.reset()
        break

env.close()
show_video()

Track generation: 999..1260 -> 261-tiles track
Reward for this episode was: -53.84615384615463, Dead at: 999
Track generation: 1113..1396 -> 283-tiles track


### Deep-Q Learning - CNN

On a closer look at the env, we noticed that LSTM may not be required, since we are not going back in track we can only move forward, hence their is no need to maintain past knowledge for that episode.


Our action space is continous as discussed above, for sake of simplicity lets try discreet space where we are going to use 5 actions only.


`A = {left, right, accelerate, decelerate, nothing}`

In [19]:
env.state.shape

(96, 96, 3)

In [49]:
from keras.models import Model
from keras.layers import Conv2D, Dense, Flatten, Input, Lambda, LSTM, Reshape
import keras.backend as K

class Qnetwork():
    def __init__(self, final_layer_size, state_shape):
        # The input image of the game is 84 x 84 x 3 (RGB)    96 x 96 x 3 (RGB)
        self.inputs = Input(shape=state_shape, name="main_input")
        print(self.inputs)
        # There will be four layers of convolutions performed on the image input
        # A convolution take a portion of an input and matrix multiplies
        # a filter on the portion to get a new input (see below)
        self.conv = Conv2D(filters=32, kernel_size=[8,8], strides=[4,4], activation="relu", padding="valid", name="conv1")(self.inputs)
        self.conv = Conv2D(filters=64, kernel_size=[4,4], strides=[2,2], activation="relu", padding="valid", name="conv2")(self.conv)
        self.conv = Conv2D(filters=64, kernel_size=[3,3], strides=[1,1], activation="relu", padding="valid", name="conv3")(self.conv)
        self.conv = Conv2D(filters= final_layer_size, kernel_size=[7,7], strides=[1,1], activation="relu",padding="valid", name="conv4")(self.conv)

        # We then separate the final convolution layer into an advantage and value
        # stream, 1/2 weights attached to each.. The value function is how well off you are in a given state.
        # The advantage is the how much better off you are after making a particular
        # move. Q is the value function of a state after a given action.
        # Advantage(state, action) = Q(state, action) - Value(state)
        
        self.stream_AC = Lambda(lambda layer: layer[:,:,:,:final_layer_size // 2],name="advantage")(self.conv)
        self.stream_VC = Lambda(lambda layer: layer[:,:,:,final_layer_size // 2:],name="value")(self.conv)

        # We then flatten the advantage and value functions
        self.stream_AC = Flatten(name="advantage_flatten")(self.stream_AC)
        self.stream_VC = Flatten(name="value_flatten")(self.stream_VC)

        ''' Now Value based model cannot be a good model as we will have to select a best action which gives maximum Q value,
        but we are not selecting actions as they have continous value, so lets try policy n/w '''

        # We define weights for our advantage and value layers. We will train these
        # layers so the matmul will match the expected value and advantage from play
        self.Advantage = Dense(5, name="advantage_final")(self.stream_AC)  # Give the advantage value of each action ?
        self.Value = Dense(1, name="value_final")(self.stream_VC)  # Give Q value of current state ? 
        
        # To get the Q output, we need to add the value to the advantage.
        # The advantage to be evaluated will be based on how good the action is based on the average advantage of that state.
        self.model = Lambda(lambda x: x[0] + (x[1] - K.mean(x[1], axis=1, keepdims=True)), name="final_out")([self.Value, self.Advantage])

        self.model = Model(self.inputs, self.model)
        self.model.compile("adam", "mse")
        self.model.optimizer.lr = 0.0001

a= = Qnetwork(512, [96, 96, 3])
a.model.summary()

Tensor("main_input_45:0", shape=(?, 96, 96, 3), dtype=float32)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 23, 23, 32)   6176        main_input[0][0]                 
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 10, 10, 64)   32832       conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 8, 8, 64)     36928       conv2[0][0]                      
_____________________________

In [0]:
def update_target_graph(main_graph, target_graph, tau):
    updated_weights = (np.array(main_graph.get_weights()) * tau) + (np.array(target_graph.get_weights()) * (1 - tau))
    target_graph.set_weights(updated_weights)
    
class ExperienceReplay:
    def __init__(self,buffer_size=50000):
        """ Data structure used to hold game experiences """
        # Buffer will contain [state,action,reward,next_state,done]
        self.buffer = []
        self.buffer_size = buffer_size
    
    def add(self, experience):
        """ Adds list of experiences to the buffer """
        # Extend the stored experiences
        self.buffer.extend(experience)
        # Keep the last buffer_size number of experiences
        self.buffer = self.buffer[-self.buffer_size:]
        
    def sample(self, size):
        """ Returns a sample of experiences from the buffer """
        sample_idxs = np.random.randint(len(self.buffer), size=size)
        sample_output = [self.buffer[idx] for idx in sample_idxs]
        sample_output = np.reshape(sample_output, (size, -1))
        return sample_output


In [10]:
import os
import gym
import numpy as np

class DeepQModel():
  
  def __init__(self, model_path):
    # Reset everything
    K.clear_session()
    self.final_layer_size = 512 #Size of the final convolution layer before splitting into Advantage and Value streams

    self.env = gym.make("CarRacing-v0")
    self.env.reset()
    
    print(self.env.observation_space.shape)
    
    # Setup our Q-networks
    self.main_qn = Qnetwork(self.final_layer_size, self.env.observation_space.shape)
    self.target_qn = Qnetwork(self.final_layer_size, self.env.observation_space.shape)

    # Make the networks equal
    update_target_graph(self.main_qn.model, self.target_qn.model, 1)

    # Setup our experience replay
    self.experience_replay = ExperienceReplay()

    self.load_model = False # Whether to load a saved model
    self.model_path = model_path
    self.main_weights_file = model_path + "/main_weights.h5" # File to save our main weights to
    self.target_weights_file = model_path + "/target_weights.h5" # File to save our target weights to

    self.prob_random_start = 0.6 # Starting chance of random action
    self.prob_random_end = 0.1 # Ending chance of random action
    
    self.y = 0.99 # Discount factor
    self.tau = 1 # Rate to update target network toward primary network
    self.update_freq = 5 # How often you update the network

    self.num_steps = [] # Tracks number of steps per episode
    self.rewards = [] # Tracks rewards per episode
    self.total_steps = 0  # Tracks cumulative steps taken in training
    
  def load_models(self):
    if self.load_model == True:
        if os.path.exists(self.main_weights_file):
            print("Loading main weights")
            self.main_qn.model.load_weights(self.main_weights_file)

        if os.path.exists(self.target_weights_file):
            print("Loading target weights")
            self.target_qn.model.load_weights(self.target_weights_file)

  def build_experiance_buffer(self, num_episode, num_episodes, steps, prob_random):
    
      # Create an experience replay for the current episode
      episode_buffer = ExperienceReplay()

      # Get the game state from the environment
      state = self.env.reset()

      done = False # Game is complete
      sum_rewards = 0 # Running sum of rewards in episode

      ''' Save all the interaction with Env '''
      for cntr in range(steps):
        if cntr >= steps or done:
          break

        self.total_steps += 1

        if num_episode / float(num_episodes) < 0.10  or np.random.rand() < prob_random:
            # Act randomly based on prob_random or if we have not accumulated enough pre_train episodes
            action = np.array(get_random_action())
        else:
            # Decide what action to take from the Q network
            action = np.eye(5)[np.argmax(self.main_qn.model.predict(np.array([state]))[0])]

        # Take the action and retrieve the next state, reward and done
        state_next, reward, done, _ = self.env.step(action)

        # Setup the episode to be stored in the episode buffer
        episode = np.array([[state], np.argmax(action), reward, [state_next], done])
        episode = episode.reshape(1,-1)

        # Store the experience in the episode buffer
        episode_buffer.add(episode)

        # Update the running rewards
        sum_rewards += reward

        # Update the state
        state = state_next

      self.experience_replay.add(episode_buffer.buffer)
      self.num_steps.append(cntr)
      self.rewards.append(sum_rewards)

#   def run_final_model(self):

#     os.system("rm -rf models_1")
#     env = wrap_env(gym.make("CarRacing-v0"))
#     state = env.reset()

#     steps = 1000
#     reward_sum = 0
#     for step in range(steps):
#         env.render()
#         action = np.eye(5)[np.argmax(self.main_qn.model.predict(np.array([state]))[0])]
#         state, reward, done, _ = env.step(action)
#         reward_sum += reward

#         if done:
#             print("Reward for this episode was: {}, Dead at: {}".format(reward_sum, step))
#             reward_sum = 0
#             env.reset()
#             break

#     env.close()
#     show_video()    

  def execute_episodes(self, num_episodes=100, batch_size=64, steps=1000, num_epochs=20, goal=5, save_every=5, print_every=5):

    # We'll begin by acting complete randomly. As we gain experience and improve,
    # we will begin reducing the probability of acting randomly, and instead
    # take the actions that our Q network suggests

    annealing_steps = num_episodes
    
    prob_random = self.prob_random_start
    prob_random_drop = (self.prob_random_start - self.prob_random_end) / annealing_steps

    losses = [0]     # Tracking training losses

    # Setup path for saving
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    for num_episode in range(num_episodes):
        self.build_experiance_buffer(num_episode, num_episodes, steps, prob_random)

        
        if prob_random > self.prob_random_end:
            # Drop the probability of a random action
            prob_random -= prob_random_drop

        if num_episode % self.update_freq == 0:
            for num_epoch in range(num_epochs):

                ''' Below Bellman equation will run we will use actual experiances from main model and
                will use actual reward + discounted rewards from target model to recalculate value

                target_q [action] = actual_reward (main_nw) + discounted_Qvalue (target_nw) 

                '''
                
                # Train batch is [[state,action,reward,next_state,done],...]
                train_batch = self.experience_replay.sample(batch_size)

                # Separate the batch into its components
                state_m, action_m, reward, state_next_m, done = train_batch.T

                # Convert the action array into an array of ints so they can be used for indexing
                action_m = action_m.astype(np.int)

                # Stack the states and train_next_state for learning
                state_m = np.vstack(state_m)
                state_next_m = np.vstack(state_next_m)

                ''' 
                In order to train n/w we need output from our target n/w modified with optimum rewards
                Based on Bellmon Equation we know that

                Q_out = rewards + discount_factor  * (future_rewards / advantage)

                We have rewards and duscount_factor, but advantage are unknown, advantage are just new state values from our target n/w.
                '''
                output_t = self.target_qn.model.predict(state_m)

                # The Q values from our target network from the next state
                output_state_next_m = self.main_qn.model.predict(state_next_m)

                action_state_next_m = np.argmax(output_state_next_m,axis=1).astype(np.int)

                # Q value of the next state based on action
                Qout_state_next_m = output_state_next_m[range(batch_size), action_state_next_m]

                ''''Tells us whether game over or not, We will multiply our rewards by this value to ensure we don't train on the last move '''
                train_gameover = done == 0

                # Reward from the action chosen in the train batch
                actual_reward = reward + (self.y * Qout_state_next_m * train_gameover)
                ''' Remember we are not overriding every value in output_t only the best action q value'''
                output_t[range(batch_size), action_m] = actual_reward
                print("****************************************")
                print(reward)
                print("========================================")
                print(actual_reward)

                # Train the main model
                '''
                Ideally we dont need target model, but here we can see that with each update weight will change
                which will contradict with target_q value we will be generating, hence we are keeping target constant but generating from 
                identical but slightly outdated same model copy.
                '''
                loss = self.main_qn.model.train_on_batch(state_m, output_t)  
                losses.append(loss)

            # Update the target model with values from the main model
            update_target_graph(self.main_qn.model, self.target_qn.model, self.tau)

            if (num_episode + 1) % save_every == 0:
                # Save the model
                self.main_qn.model.save_weights(self.main_weights_file)
                self.target_qn.model.save_weights(self.target_weights_file)

        if num_episode % print_every == 0:
            # Print progress
            mean_loss = np.mean(losses[-(print_every * num_epochs):])

            print("Num episode: {} Mean reward: {:0.4f} Prob random: {:0.4f}, Loss: {:0.04f}".format(
                num_episode, np.mean(self.rewards[-print_every:]), prob_random, mean_loss))
            if np.mean(self.rewards[-print_every:]) >= goal:
                print("Training complete!")
                break
             
'''
batch_size - How many Experiences to use for each training step
steps = 1000 # Maximum allowed episode length
num_epochs = 20 # How many epochs to train from single episode experiances

save_every = 5   # How often to save
print_every = 5 # How often to print status
'''

model_path = "./models_1"
!rm -rf models_1

obj = DeepQModel(model_path)
obj.execute_episodes(num_episodes=4, batch_size=64, steps=1000, num_epochs=20, goal=900, save_every=10, print_every=2)
# !rm -rf models_1
# obj.run_final_model()

Track generation: 1012..1269 -> 257-tiles track
(96, 96, 3)
Track generation: 1023..1282 -> 259-tiles track
****************************************
[-0.10000000000000009 3.775968992248062 -0.10000000000000142
 -0.10000000000000142 -0.09999999999999964 -0.10000000000000142
 -0.10000000000000142 -0.09999999999999964 -0.09999999999999964
 -0.10000000000000142 -0.10000000000000142 -0.10000000000000142
 -0.09999999999999964 3.7759689922480617 -0.09999999999999964
 -0.09999999999999964 -0.09999999999999964 -0.10000000000000142
 -0.10000000000000142 -0.10000000000000142 -0.09999999999999964
 -0.10000000000000142 -0.10000000000000142 -0.10000000000000142
 -0.10000000000000009 -0.09999999999999964 -0.10000000000000142
 -0.09999999999999964 -0.09999999999999964 -0.10000000000000142 -0.1
 -0.09999999999999964 -0.10000000000000142 -0.10000000000000142
 -0.10000000000000142 -0.10000000000000142 -0.10000000000000142
 -0.10000000000000142 -0.09999999999999964 -0.1 -0.10000000000000142
 -0.1000000000

In [8]:
def run_final_model(obj):
  os.system("rm -rf models_1")
  env = wrap_env(gym.make("CarRacing-v0"))
  state = env.reset()

  steps = 1000
  reward_sum = 0
  for step in range(steps):
      env.render()
      action = np.eye(5)[np.argmax(obj.main_qn.model.predict(np.array([state]))[0])]
      state, reward, done, _ = env.step(action)
      reward_sum += reward

      if done:
          print("Reward for this episode was: {}, Dead at: {}".format(reward_sum, step))
          reward_sum = 0
#           env.reset()
          break
  env.close()
  show_video()

run_final_model(obj)

Track generation: 1154..1451 -> 297-tiles track
Reward for this episode was: -62.83783783783861, Dead at: 999
